<a href="https://colab.research.google.com/github/JHyunjun/Attention/blob/main/TF2.0_AttentionGRU_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.03.08

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
#xy_test = xy_train

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

xy_train = MinMaxScaler(xy_train)
xy_test = MinMaxScaler(xy_test)


In [ ]:
# Data Sequence Setting

TIME_STEPS = 5
seq_length = TIME_STEPS

train_set = xy_train
test_set = xy_test

def build_dataset(time_series, seq_length) : 
  dataX = []
  dataY = []
  for i in range(0, len(time_series) -seq_length) : 
    x = time_series[i:i+seq_length , : :]
    y = time_series[i + seq_length -1, :]
    dataX.append(x)
    dataY.append(y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print(trainX.shape, trainY.shape) #(# of datas, Time_steps, features)

INPUT_DIM = trainX.shape[2]

In [ ]:
#AttentionGRUAE NEW

latent_dim = int((INPUT_DIM * TIME_STEPS)/10) #(5 * 20)/10 = 10

def attention_layer_encoder(inputs) :
  input_dim = int(inputs.shape[2])
  a = tf.keras.layers.Permute((2,1))(inputs)
  a = tf.keras.layers.Dense(TIME_STEPS, activation = 'softmax')(a)
  a_probs = tf.keras.layers.Permute((2,1), name = 'attention_vec_encoder')(a)
  output_attention_mul = tf.keras.layers.multiply([inputs, a_probs])

  return output_attention_mul

def attention_layer_decoder(inputs) :
  input_dim = int(inputs.shape[2])
  a = tf.keras.layers.Permute((2,1))(inputs)
  a = tf.keras.layers.Dense(TIME_STEPS, activation = 'softmax')(a)
  a_probs = tf.keras.layers.Permute((2,1), name = 'attention_vec_decoder')(a)
  output_attention_mul = tf.keras.layers.multiply([inputs, a_probs])

  return output_attention_mul

def Attention_GRU_AE() : 
  # Encoding
  inputs = tf.keras.Input(shape = (TIME_STEPS, INPUT_DIM, ))
  print("A : ", inputs.shape)
  gru_units = 32
  gru_out = tf.keras.layers.GRU(gru_units, return_sequences = True)(inputs)
  print("B : ", gru_out.shape)
  gru_out = tf.keras.layers.GRU(gru_units, return_sequences = True)(gru_out)
  print("C : ", gru_out.shape)
  encoder_output = attention_layer_encoder(gru_out)
  print("D : ", encoder_output.shape)
  # Decoding
  decoder_gru_units = 32
  decoder_gru_out = tf.keras.layers.GRU(decoder_gru_units, return_sequences = True)(encoder_output)
  print("G : ",  decoder_gru_out.shape)
  decoder_gru_out = tf.keras.layers.GRU(decoder_gru_units, return_sequences = True)(decoder_gru_out)
  print("H : ",  decoder_gru_out.shape)
  attention_decoder = attention_layer_decoder(decoder_gru_out)
  print("I : ",  attention_decoder.shape)
  attention_decoder = tf.keras.layers.Flatten()(attention_decoder)
  print("J : ",  attention_decoder.shape)
  output = tf.keras.layers.Dense(INPUT_DIM, activation = 'sigmoid')(attention_decoder)
  print("K : ",  output.shape)
  model = tf.keras.Model(inputs = [inputs], outputs = output)

  return model

In [ ]:
#RUN
modelhj = Attention_GRU_AE()
modelhj.compile(optimizer = 'adam', loss = 'mse')
modelhj.fit(trainX, trainY, epochs = 3, batch_size = 1, validation_split = 0)

In [ ]:
# Training
trainY_predict = modelhj.predict(trainX)
print(trainY_predict.shape)

In [ ]:
trainY_predict_0 = trainY_predict[:,0]
trainY_predict_1 = trainY_predict[:,1]
trainY_predict_2 = trainY_predict[:,2]
trainY_predict_3 = trainY_predict[:,3]
trainY_predict_4 = trainY_predict[:,4]

#Figure
plt.figure(figsize = (6,8))
plt.plot(trainY[seq_length:,0], color = 'blue')
plt.plot(trainY_predict_0, color = 'red')
plt.legend(['Real Current','Predicted Current'])

#Figure
plt.figure(figsize = (6,8))
plt.plot(trainY[seq_length:,1], color = 'blue')
plt.plot(trainY_predict_1, color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

#Figure
plt.figure(figsize = (6,8))
plt.plot(trainY[seq_length:,2], color = 'blue')
plt.plot(trainY_predict_2, color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

#Figure
plt.figure(figsize = (6,8))
plt.plot(trainY[seq_length:,3], color = 'blue')
plt.plot(trainY_predict_3, color = 'red')
plt.legend(['Real Current','Predicted Current'])

#Figure
plt.figure(figsize = (6,8))
plt.plot(trainY[seq_length:,4], color = 'blue')
plt.plot(trainY_predict_4, color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])